In [ ]:
import cv2
import numpy as np
import pandas as pd

from pathlib import Path

from dewan_calcium.helpers import DewanEPM

In [ ]:
data_dir = Path('C:\Projects\EPM_Data') 

video_path = data_dir.joinpath('video.mkv')
points_path = data_dir.joinpath('points.h5')
image_path = data_dir.joinpath('labeled_image.jpg')

In [ ]:
points = pd.read_hdf(points_path) # Read the points
cols = ['mouse_x', 'mouse_y', 'mouse_p', 'led_x', 'led_y', 'led_p']  # Reset the column names to something sensible
points.columns = cols 

# There may be an instance where the model erroneously identified the LED for very short time periods
# find_led_start bins the possible LED on times (anywhere led_p > 0.98)
# We then find the bin with the largest size, which means it has the most frames where the LED is identified
# This is most likely the period where the experimenter turned on the LED
led_bins = np.array(DewanEPM.find_led_start(points))
true_led_bin = np.argmax(np.subtract(led_bins[:, 1], led_bins[:,0]))

led_on = led_bins[true_led_bin][0] # Find first row where the LED is 'on'
good_points = points.iloc[led_on:-1] # Delete all data before the LED is 'on'
good_points.reset_index(drop=True, inplace=True) # Reset the index

# Get X, Y coordinates, cast to int, and combine them into tuples
head_x = good_points['mouse_x'].astype(int)
head_y = good_points['mouse_y'].astype(int)
coordinates = list(zip(head_x, head_y))

In [ ]:
video = cv2.VideoCapture(str(video_path))  # Load Video
_ = video.set(cv2.CAP_PROP_POS_FRAMES, led_on)  # Set the first frame to the first LED frame
_, frame = video.read() # Read said frame

for pair in coordinates:  # Loop through each coordinate and draw a red point
    cv2.circle(frame, pair, 0, (0,0,255), -1)

_ = cv2.imwrite(str(image_path), frame) # Save image